# Free Fall

### **Introduction**

Here, on Earth objects are constantly fall down. Nowadays, students learn about the constant of gravitation (G) and the gravitational acceleration (g) already at school, however, humanity calculated this constants and the equations describing such type motion as falling comperatively recently.

### **What is it?**

Free fall is a state at which an object is subjected to the effect of gravitation, that leads to the fall of such object. You might be deceived by the fact that only the heavier objects pull the lighter ones, however, while say Earth pulls the moon closer, the Moon also pulls the Earth closer. So why does the Moon orbit the Earth then and not vice versa? The force of the pull matters, the force projected onto the Moon is drastically larger than that of force projected onto the Earth.

### **Example**

What will fall faster, a hummer or a feather? If you have not been asked it before, your most probable answer will be the hummer. But the answer is: "it depends". But what is the dependency? If you apply the formula for the falling object: $y=\frac{1}{2}gt^2$, you will say that the hummer and the feather are going to fall down at the same time, because the equation does not take into account the mass of the object. So why then the hummer falls faster? The answer is the air. Air has density which slows down objects and the smaller the mass and bigger the sufrace of the object the slower it falls. In fact, on Earth, obejcts may reach their maximum velocity called the **terminal velocity**. The feather falls slower than the hummer because it reaches its terminal velocity almost instantly, that means it falls with a constant velocity. In vacuum though, these two objects will fall down as the equation describes. Watch the [video](https://www.youtube.com/watch?v=Oo8TaPVsn9Y) below, where astranauts make this experiment on the Moon (in vacuum, with no air).

### Video of falling object on the Moon (in vacuum)

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo("Oo8TaPVsn9Y", width=512)

In this lab you will experiment with the falling objects tweaking the environment (vacuum or not) and the masses of the objects.

# Simulation

## Setup

Installing packages (for Google Colab). If this notebook is opened in Google Colab then some packages must be installed to run the code!\
Then importing the scene and plot settings.

In [ ]:
#@title Run to install MuJoCo and `dm_control` for Google Colab

IS_COLAB = 'google.colab' in str(get_ipython())
if IS_COLAB:
    # download the repository
    !git clone https://github.com/commanderxa/alphalabs.git
    from alphalabs.mechanics.setup import install_packages_colab
    install_packages_colab()
    import alphalabs.mechanics.plot
    from alphalabs.mechanics.scene import Scene
else:
    import os, sys
    module_path = os.path.abspath(os.path.join(".."))
    if module_path not in sys.path:
        sys.path.append(module_path)
    # import the scene
    from scene import Scene
    import plot

## Import

Import all required packages to preform simulations. Packages include simulation engine, plotting libraries and other ones necessary for computations.

In [ ]:
%env MUJOCO_GL=egl

# simulation
from dm_control import mjcf

# for video recording
import mediapy

# computations
import numpy as np

# plot charts
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Initial Conditions

In this block constants are defined. They impact the environment, rendering and objects directly.

In [ ]:
# global
viscosity = 0.00002  # Air Resistance
density = 1.2

# simulation constants
blue_mass = 0.1  # mass [kg]
red_mass = 100
distance = 20  # [m]
is_vacuum = False

# rendering
width = 1920
height = 1080
dpi = 600
duration = 30  # (seconds)
framerate = 60  # (Hz)

## Model

### Objects of Interest

This class defines the object of our interest, a `box`. Here we write what is this object (box), what can it do (move, fall) and also add a camera that follows the object.

In [ ]:
class Sphere(object):

    def __init__(self, name: str, size: float, mass: float, rgba: list[float]) -> None:
        self.model = mjcf.RootElement(model=name)

        self.sphere = self.model.worldbody.add("body", name="sphere", pos=[0, 0, 0])
        self.sphere_geom = self.sphere.add(
            "geom",
            name="sphere_geom",
            type="sphere",
            size=[size],
            rgba=rgba,
            mass=mass,
        )

        self.fall = self.sphere.add("joint", name="fall", type="slide", axis=[0, 0, 1])

### World Model

Collecting everything into one general model

In [ ]:
class Model(object):

    def __init__(self, blue_mass: float, red_mass: float, distance: float) -> None:
        self.model = mjcf.RootElement(model="model")

        sphere_radius = 0.5

        # set render info
        self.model.visual.__getattr__("global").offheight = height
        self.model.visual.__getattr__("global").offwidth = width

        # set the simulation constants
        if not is_vacuum:
            self.model.option.viscosity = viscosity
            self.model.option.density = density

        self.model.option.integrator = "RK4"
        self.model.option.timestep = 0.001

        # create the scene (ground)
        self.scene = Scene(length=10, width=8)
        self.scene_site = self.model.worldbody.add("site", pos=[0, 0, -sphere_radius])
        self.scene_site.attach(self.scene.model)

        self.spheres = self.model.worldbody.add("body", name="spheres", pos=[0, 0, distance])
        self.camera = self.spheres.add(
            "camera",
            name="front",
            pos=[0, -10, 0],
            euler=[90, 0, 0],
            mode="trackcom",
        )

        # add sphere
        self.sphere = Sphere(
            name="blue_sphere", size=sphere_radius, mass=blue_mass, rgba=[0.2, 0.3, 0.9, 1]
        )
        sphere_site = self.spheres.add("site", pos=[-1, 0, 0])
        sphere_site.attach(self.sphere.model)

        # add sphere
        self.sphere2 = Sphere(
            name="red_sphere", size=sphere_radius, mass=red_mass, rgba=[0.9, 0.2, 0.3, 1]
        )
        sphere_site2 = self.spheres.add("site", pos=[1, 0, 0])
        sphere_site2.attach(self.sphere2.model)

## Simulation

Initializing the `physics` of the simulation

In [ ]:
model = Model(blue_mass, red_mass, distance).model
physics = mjcf.Physics.from_mjcf_model(model)

First of all, the environment must be verified by rendering a picture

In [ ]:
mediapy.show_image(physics.render(height=320, width=640, camera_id=0))

Get IDs of interesting objects

In [ ]:
ground_id = physics.model.name2id("scene/platform", "geom")
blue_id = physics.model.name2id("blue_sphere/sphere_geom", "geom")
red_id = physics.model.name2id("red_sphere/sphere_geom", "geom")
ground_id, blue_id, red_id

### Simulation Loop

In [ ]:
frames = []
timevals = []

blue_velocity = []
blue_position = []

red_velocity = []
red_position = []

stop = None
while physics.time() < duration:
    if stop is not None and physics.data.time - stop > 1:
        break

    blue_stop = None
    red_stop = None
    for i, c in enumerate(physics.data.contact):
        if red_id in c.geom and ground_id in c.geom and stop is None:
            red_stop = physics.data.time
        if blue_id in c.geom and ground_id in c.geom and stop is None:
            blue_stop = physics.data.time
    if blue_stop is not None and red_stop is not None:
        stop = physics.time()

    timevals.append(physics.time())

    blue_velocity.append(physics.named.data.qvel["blue_sphere/fall"][0].copy())
    blue_position.append(
        physics.named.data.geom_xpos["blue_sphere/sphere_geom"][-1].copy()
    )

    red_velocity.append(physics.named.data.qvel["red_sphere/fall"][0].copy())
    red_position.append(
        physics.named.data.geom_xpos["red_sphere/sphere_geom"][-1].copy()
    )

    if len(frames) < physics.time() * framerate:
        pixels = physics.render(width=width, height=height, camera_id=0)
        frames.append(pixels)

    physics.step()

In [ ]:
len(frames)

In [ ]:
mediapy.show_video(frames, fps=framerate)

In [ ]:
video_name = f"free_fall" if IS_COLAB else f"../../output/free_fall"
mediapy.write_video(video_name + ".mp4", images=frames, fps=framerate)

## Simulation Data Visualization

Convert data into numpy array to have more features

In [ ]:
blue_velocity = np.array(blue_velocity)
red_velocity = np.array(red_velocity)

blue_position = np.array(blue_position)
red_position = np.array(red_position)

In [ ]:
cm = 1 / 2.54
figsize = (8.5 * cm, 3.75 * cm)
fig, ax = plt.subplots(ncols=2, figsize=figsize, dpi=dpi)
fig.subplots_adjust(wspace=1 * cm)

y_labels = ["Amplitude [m/s]", "Height, [m]"]
x_labels = ["Time, [s]", "Time, [s]"]
titles = ["Velocity", "Position"]
data = [[blue_velocity, red_velocity], [blue_position, red_position]]

for i in range(2):
    sns.lineplot(x=timevals, y=data[i][0], ax=ax[i], linewidth=1, label="Blue Sphere", color="blue")
    sns.lineplot(x=timevals, y=data[i][1], ax=ax[i], linewidth=1, label="Red Sphere", color="red")
    ax[i].set_title(titles[i], fontsize=8)
    ax[i].set_ylabel(y_labels[i], fontsize=7, labelpad=2)
    ax[i].set_xlabel(x_labels[i], fontsize=7, labelpad=2)
    ax[i].tick_params(labelsize=6)
    ax[i].tick_params(which="both", direction="in", top=True, right=True, length=2)
    ax[i].legend(fontsize=5, handlelength=1).get_frame().set_linewidth(0.5)

chart_name = f"free_fall" if IS_COLAB else f"../../output/free_fall"
fig.savefig(chart_name + ".png", bbox_inches="tight")